In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

In [96]:
df = pd.read_csv(r"datasets/clean_data.csv")

In [97]:
v = df["vote_count"]
C = df["vote_average"].mean()
R = df["vote_average"]

In [98]:
# m = 1900
m = int(df["vote_count"].quantile(0.9))

In [99]:
def weighted_avg_rating(x, m=m, C=C):
    v = x["vote_count"]
    R = x["vote_average"]

    return ((R * v) + (C * m)) / (v + m)

In [100]:
data = df[df["vote_count"] >= m].copy()

In [101]:
data["weighted_avg"] = data.apply(weighted_avg_rating, axis=1)

In [102]:
df_sorted_ranking = data.sort_values(by="weighted_avg", ascending=False)

In [103]:
popularity = data.sort_values(by="popularity", ascending=False)

In [104]:
scaling = MinMaxScaler()

In [105]:
popularity.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline',
       'vote_average', 'vote_count', 'cast', 'crew', 'weighted_avg'],
      dtype='object')

In [106]:
popularity[["weighted_avg", "popularity"]]

,weighted_avg,popularity
546,6.311720,875.581305
95,7.809533,724.247784
788,7.212687,514.569956
94,7.613058,481.098624
127,7.019246,434.278564
...,...,...
101,6.836089,3.195174
203,6.556552,2.871739
508,6.154176,2.502487
2511,6.664349,2.186927


In [107]:
df_scaled = scaling.fit_transform(popularity[["weighted_avg", "popularity"]])

In [108]:
popularity[["weighted_avg_scaled", "popularity_scaled"]] = df_scaled

In [109]:
popularity["score_mix"] = (
    popularity["weighted_avg_scaled"] * 0.5 + popularity["popularity_scaled"] * 0.50
)

In [110]:
popularity_sorted = popularity.sort_values(by="score_mix", ascending=False)

In [126]:
px.bar(
    popularity_sorted.head(10),
    x="original_title",
    y="score_mix",
    labels={
        "original_title": "Movie Title",
        "score_mix": "Weighted Popularity Score",
    },
    title="Top 10 Movies by Weighted Popularity",
    width=1200,
    height=500,
    color="original_title",
)